<a href="https://colab.research.google.com/github/preetamnahak007/deep-learning-tensorflow/blob/master/word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow-gpu==1.13.1

     |████████████████████████████████| 345.2MB 48kB/s 
     |████████████████████████████████| 368kB 38.0MB/s 
     |████████████████████████████████| 3.2MB 38.9MB/s 
ERROR: tensorflow 2.2.0rc4 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 1.13.1 which is incompatible.
ERROR: tensorflow 2.2.0rc4 has requirement tensorflow-estimator<2.3.0,>=2.2.0, but you'll have tensorflow-estimator 1.13.0 which is incompatible.
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: tensorboard 2.2.1
    Uninstalling tensorboard-2.2.1:
      Successfully uninstalled tensorboard-2.2.1


In [3]:
!pip install tensorflow

     |████████████████████████████████| 3.0MB 4.7MB/s 
     |████████████████████████████████| 460kB 35.4MB/s 
ERROR: tensorflow-gpu 1.13.1 has requirement tensorboard<1.14.0,>=1.13.0, but you'll have tensorboard 2.2.1 which is incompatible.
ERROR: tensorflow-gpu 1.13.1 has requirement tensorflow-estimator<1.14.0rc0,>=1.13.0, but you'll have tensorflow-estimator 2.2.0 which is incompatible.
  Found existing installation: tensorboard 1.13.1
    Uninstalling tensorboard-1.13.1:
      Successfully uninstalled tensorboard-1.13.1
  Found existing installation: tensorflow-estimator 1.13.0
    Uninstalling tensorflow-estimator-1.13.0:
      Successfully uninstalled tensorflow-estimator-1.13.0


In [1]:
import numpy as np
import tensorflow as tf

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [0]:
corpus_raw = 'He is the king . The king is royal . She is the royal  queen '
# convert to lower case
corpus_raw = corpus_raw.lower()

In [0]:
words = []
for word in corpus_raw.split():
    if word != '.': #  don't  treat . as a word
        words.append(word)
words = set(words) #all duplicate words are removed
word2int = {}
int2word = {}
vocab_size = len(words) # gives the total number of unique words
for i,word in enumerate(words):
    word2int[word] = i
    int2word[i] = word

In [6]:
# raw sentences is a list of sentences.
raw_sentences = corpus_raw.split('.')
sentences = []
for sentence in raw_sentences:
    sentences.append(sentence.split())
print(sentences)

[['he', 'is', 'the', 'king'], ['the', 'king', 'is', 'royal'], ['she', 'is', 'the', 'royal', 'queen']]


# Preparing training data

In [12]:
#Generating our training data
data = []
WINDOW_SIZE = 3
for sentence in sentences:
    for word_index, word in enumerate(sentence):
        for nb_word in sentence[max(word_index - WINDOW_SIZE, 0) : min(word_index + WINDOW_SIZE, len(sentence)) + 1] : 
            if nb_word != word:
                data.append([word, nb_word])
print(data)

[['he', 'is'], ['he', 'the'], ['he', 'king'], ['is', 'he'], ['is', 'the'], ['is', 'king'], ['the', 'he'], ['the', 'is'], ['the', 'king'], ['king', 'he'], ['king', 'is'], ['king', 'the'], ['the', 'king'], ['the', 'is'], ['the', 'royal'], ['king', 'the'], ['king', 'is'], ['king', 'royal'], ['is', 'the'], ['is', 'king'], ['is', 'royal'], ['royal', 'the'], ['royal', 'king'], ['royal', 'is'], ['she', 'is'], ['she', 'the'], ['she', 'royal'], ['is', 'she'], ['is', 'the'], ['is', 'royal'], ['is', 'queen'], ['the', 'she'], ['the', 'is'], ['the', 'royal'], ['the', 'queen'], ['royal', 'she'], ['royal', 'is'], ['royal', 'the'], ['royal', 'queen'], ['queen', 'is'], ['queen', 'the'], ['queen', 'royal']]


In [15]:
def convert_to_one_hot(word_index, vocab_size):
  temp = np.zeros(vocab_size)
  temp[word_index] = 1
  return temp

X_train = []
Y_train = []

for word_nb_pair in data:
  X_train.append(convert_to_one_hot(word2int[ word_nb_pair[0] ],  vocab_size))
  Y_train.append(convert_to_one_hot(word2int[ word_nb_pair[1] ],  vocab_size))

# convert them to numpy arrays
X_train = np.asarray(X_train)
Y_train = np.asarray(Y_train)

print(X_train.shape, Y_train.shape)

(42, 7) (42, 7)


# Building the model

In [0]:
# making placeholders for x_train and y_train
x = tf.placeholder(tf.float32, shape=(None, vocab_size))
y_label = tf.placeholder(tf.float32, shape=(None, vocab_size))

In [17]:
EMBEDDING_DIM = 5 # can be anything => dimension of embedded vector
W1 = tf.Variable(tf.random_normal([vocab_size, EMBEDDING_DIM]))
b1 = tf.Variable(tf.random_normal([EMBEDDING_DIM])) #bias
hidden_layer = tf.add(tf.matmul(x,W1), b1) #hidden layer

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
W2 = tf.Variable(tf.random_normal([EMBEDDING_DIM, vocab_size]))
b2 = tf.Variable(tf.random_normal([vocab_size]))
prediction = tf.nn.softmax(tf.add( tf.matmul(hidden_layer, W2), b2)) #prediction/output layer

# Train the model

In [31]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init) #make sure you do this!
# define the loss function:
cross_entropy_loss = tf.reduce_mean(-tf.reduce_sum(y_label * tf.log(prediction), reduction_indices=[1]))
# define the training step:
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy_loss)
n_iters = 50000
# train for n_iter iterations
for _ in range(n_iters):
    sess.run(train_step, feed_dict={x: X_train, y_label: Y_train})
    print('loss is : ', sess.run(cross_entropy_loss, feed_dict={x: X_train, y_label: Y_train}))

Streaming output truncated to the last 5000 lines.
loss is :  1.49126
loss is :  1.4912602
loss is :  1.49126
loss is :  1.4912602
loss is :  1.4912602
loss is :  1.49126
loss is :  1.49126
loss is :  1.49126
loss is :  1.4912602
loss is :  1.4912599
loss is :  1.49126
loss is :  1.49126
loss is :  1.49126
loss is :  1.4912602
loss is :  1.4912602
loss is :  1.4912599
loss is :  1.49126
loss is :  1.49126
loss is :  1.4912602
loss is :  1.49126
loss is :  1.4912599
loss is :  1.49126
loss is :  1.49126
loss is :  1.4912599
loss is :  1.49126
loss is :  1.49126
loss is :  1.49126
loss is :  1.4912599
loss is :  1.4912602
loss is :  1.4912599
loss is :  1.4912599
loss is :  1.49126
loss is :  1.4912599
loss is :  1.49126
loss is :  1.4912599
loss is :  1.4912599
loss is :  1.4912599
loss is :  1.4912599
loss is :  1.4912599
loss is :  1.4912599
loss is :  1.49126
loss is :  1.49126
loss is :  1.49126
loss is :  1.4912599
loss is :  1.4912599
loss is :  1.4912599
loss is :  1.4912599
loss

Printing our wordEmbedding matrix

In [33]:
#Our word2vec weight matrix
vect = sess.run(W1 + b1)
print(vect)

[[-0.64111066  2.2560225   1.6108928   0.7317032   1.4030876 ]
 [-1.1418805   4.1069965  -1.8773841   0.91014975 -0.44532585]
 [ 0.4302839   1.6512817   0.511131   -0.43024534 -3.0498624 ]
 [-0.46371967  3.6087728  -0.6171703   0.82512766  0.18847984]
 [ 0.74079883  0.26191056 -0.82312524 -2.5034394   0.04884979]
 [-1.5415593   1.3926307  -2.6432722  -1.6231148  -0.7741676 ]
 [ 0.986853   -0.9055383  -0.3849998   1.8600459   0.5618466 ]]


Let's find embedded form of word 'queen'

In [34]:
print(vect[ word2int['queen'] ])

[-1.1418805   4.1069965  -1.8773841   0.91014975 -0.44532585]


# Let's verify if related words are closer or not

In [38]:
def euclidean_dist(vec1, vec2):
    return np.sqrt(np.sum((vec1-vec2)**2))

queen_vect = vect[ word2int['queen'] ]
she_vect = vect[ word2int['she'] ]

king_vect = vect[ word2int['king'] ]
he_vect = vect[ word2int['he'] ]

print(euclidean_dist(queen_vect, she_vect))
print(euclidean_dist(king_vect, he_vect))
print(euclidean_dist(queen_vect, king_vect))

1.6447512
5.4717255
4.3924284
